### Project Milestone 3
I want to use some data from wiki as a guide to compare the number of phone numbers in use vs the population as a comparison visual for my project. 
The only issue is, I have decided to focus only on the united states and need to strip out the data. 

In [2]:
# Let's start with importing the wiki data.

import pandas as pd

# Read all tables from the web page into a list of DataFrames
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_number_of_mobile_phones_in_use')

# Get the first table from the list
df = tables[0]

# Print the first 5 rows of the DataFrame
df

,Rankings,Country or regions,# of phone numbers,Population,Connections/100 citizens,Date of evaluation
0,World,World,"7,950,000,000+",7621018958,104.32,2019/12[1][2]
1,1,China,1610360000,1420050000,113.38,2020/10[3][4]
2,2,India,1515971713,1375245994,110.18,2020/10[3][5]
3,3,Indonesia,385573398,237556363,162.28,2016/07[6]
4,4,United States,380577528,327874628,116.27,2020/10[7][8]
...,...,...,...,...,...,...
72,72,Estonia,1904000,1269420,150.5,2015/07[79]
73,73,Cuba,1300000,11200000,11.6,2011/12[80]
74,74,Montenegro,1117500,620029,180.23,2015/09[81]
75,75,Malta,554651,452515,122.57,2013/06[82]


In [3]:
# now let's add the three columns I need to a variable. 
col1 = df.loc[:, 'Country or regions']
col2 = df.loc[:, '# of phone numbers']
col3 = df.loc[:, 'Population']

# Concatenate values of the three columns into a new variable
strippedData = pd.concat([col1, col2, col3], axis=1)
strippedData.head(3)

,Country or regions,# of phone numbers,Population
0,World,"7,950,000,000+",7621018958
1,China,1610360000,1420050000
2,India,1515971713,1375245994


In [6]:
import sqlite3
# Connect to the database
conn = sqlite3.connect('Projectdata.db')

# Write the DataFrame to a SQLite3 table
strippedData.to_sql(name='worlddata', con=conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [17]:
# I need the # of phone numbers and population columns as int. There is str chatacter in the second column

strippedData['# of phone numbers'] = strippedData['# of phone numbers'].str.replace('+', '')
strippedData.head(5)

C:\Users\chris\AppData\Local\Temp\ipykernel_26272\3395652052.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  strippedData['# of phone numbers'] = strippedData['# of phone numbers'].str.replace('+', '')


,Country or regions,# of phone numbers,Population
0,World,"7,950,000,000",7621018958
1,China,1610360000,1420050000
2,India,1515971713,1375245994
3,Indonesia,385573398,237556363
4,United States,380577528,327874628


In [24]:
# decided to go with str here, as the ',' in the world figure would not work in the division transformation
strippedData['# of phone numbers'] = pd.to_numeric(strippedData['# of phone numbers'], errors='coerce').astype(str)
strippedData.head(5)

,Country or regions,# of phone numbers,Population
0,World,nan,7621018958
1,China,1610360000.0,1420050000
2,India,1515971713.0,1375245994
3,Indonesia,385573398.0,237556363
4,United States,380577528.0,327874628


In [21]:
# going to try and replace 'nan' with 0
# Did not work :(
strippedData.fillna(0, inplace=True)
strippedData.head(5)

,Country or regions,# of phone numbers,Population
0,World,nan,7621018958
1,China,1610360000.0,1420050000
2,India,1515971713.0,1375245994
3,Indonesia,385573398.0,237556363
4,United States,380577528.0,327874628


In [26]:
# Decided to count number of nan values and then remove rows
# Still there, can't get rows with nan to be removed from data frame

strippedData.dropna(inplace=True)
strippedData.head(5)

,Country or regions,# of phone numbers,Population
0,World,nan,7621018958
1,China,1610360000.0,1420050000
2,India,1515971713.0,1375245994
3,Indonesia,385573398.0,237556363
4,United States,380577528.0,327874628


In [28]:
# I only need the United States row, so I am going to strip it to a new variable. 
USdata = strippedData.loc[4, :].to_frame().T
USdata

,Country or regions,# of phone numbers,Population
4,United States,380577528.0,327874628


In [32]:
# Going to try and convert the numbers to int again to see if I can divide the columns later. 

USdata['# of phone numbers'] = pd.to_numeric(USdata['# of phone numbers'], errors='coerce').astype(int)
USdata

,Country or regions,# of phone numbers,Population
4,United States,380577528,327874628


In [42]:
# Now I am going to try and get the percentage of population vs phone nubmers in use. 
# Finally worked!

USdata['Percentage'] = USdata['# of phone numbers'] / USdata['Population'].sum()*100
USdata

,Country or regions,# of phone numbers,Population,Percentage
4,United States,380577528,327874628,116.074101


In [46]:
# I'm curious what our outliers look like

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = strippedData[~((strippedData >= lower_bound) & (strippedData <= upper_bound)).all(axis=1)]

outliers.head(5)

C:\Users\chris\AppData\Local\Temp\ipykernel_26272\1999970393.py:9: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = strippedData[~((strippedData >= lower_bound) & (strippedData <= upper_bound)).all(axis=1)]


,Country or regions,# of phone numbers,Population
0,World,nan,7621018958
1,China,1610360000.0,1420050000
2,India,1515971713.0,1375245994
3,Indonesia,385573398.0,237556363
4,United States,380577528.0,327874628


In [48]:
# The '#' in the header doesn't format well for project charts. Remaming column header
strippedData= strippedData.rename(columns={'# of phone numbers': 'Count Numbers in Use'})
strippedData.head(2)

,Country or regions,Count Numbers in Use,Population
0,World,nan,7621018958
1,China,1610360000.0,1420050000


In [49]:
# I need one more data set for my project. Importing percetage of population owning a smartphone

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_smartphone_penetration')

# Get the first table from the list
df5 = tables[0]

# Print the first 5 rows of the DataFrame
df5.head(5)

,Rank,Country/Region,Total population,Smartphone users,Smartphone penetration
0,1,PRC,1.44B,865.04M,59.9%
1,2,India,1.39B,606.57M,43.5%
2,3,United States,332.92M,240.15M,72.1%
3,4,Indonesia,276.36M,178.96M,64.8%
4,6,Russia,145.91M,102.24M,70.1%


In [52]:
# I would like to order the data by the penetration column
df5.sort_values(by='Smartphone penetration', ascending=True)
df5.head(5)


,Rank,Country/Region,Total population,Smartphone users,Smartphone penetration
0,1,PRC,1.44B,865.04M,59.9%
1,2,India,1.39B,606.57M,43.5%
2,3,United States,332.92M,240.15M,72.1%
3,4,Indonesia,276.36M,178.96M,64.8%
4,6,Russia,145.91M,102.24M,70.1%
